In [1]:
from src.turbofan import Turbofan
from src.turboprop import Turboprop
from utils.configs import (
    config_ex71,
    config_ex72,
    config_ex73,
)

In [2]:
turboprop = Turboprop(config_ex71)
turboprop.set_sea_level_air_flow(8.49)
turboprop.print_outputs()
turboprop.calibrate_pot_th()
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 288.150              | 101.300             
3 (Compressor)       | 713.156              | 1597.501            
4 (Câmara)           | 1600.000             | 1597.501            
5 (Turbina Comp.)    | 1174.994             | 390.159             
6 (Turbina Livre)    | 931.033              | 135.575             

[ Dados da Turbina Livre ]
Razão de expansão na tl            : 2.878
Pressão total na saída da tl       : 135.58 kPa
Temperatura Isentrópica Total      : 903.93 K
Temperatura Total                  : 931.03 K
Cp_tl                              : 1.16
Trabalho Isentrópico               : 314.44 kJ/kg
Trabalho Real                      : 282.99 kJ/kg
Vazão na turbina                   : 8.68 kg/s
Potência na Turbina Livre          : 2457.33 kW
Potência na Gearbox

### Exemplo 7.2

In [3]:
turboprop.update_final_config(config_ex72)
turboprop.update_from_N2(0.85)
turboprop.print_outputs()

--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação              | Temp. Total (K)      | Pressão Total (kPa) 
------------------------------------------------------------------
2 (Inlet)            | 256.535              | 46.156              
3 (Compressor)       | 577.216              | 534.552             
4 (Câmara)           | 1310.669             | 534.552             
5 (Turbina Comp.)    | 989.988              | 144.472             
6 (Turbina Livre)    | 798.312              | 55.727              

[ Dados da Turbina Livre ]
Razão de expansão na tl            : 2.593
N2                                 : 0.850
Pressão total na saída da tl       : 55.73 kPa
Temperatura Isentrópica Total      : 781.59 K
Temperatura Total                  : 798.31 K
Cp_tl                              : 1.16
Trabalho Isentrópico               : 241.74 kJ/kg
Trabalho Real                      : 222.34 kJ/kg
Vazão na turbina                   : 2.85 kg/s
Potência na Turbina Livre

In [4]:
turboprop.print_config()


--- Configuração do Motor Turboprop ---

[Condições de Voo e Ambiente]
mach                    : 0.450
altitude                : 0.000
t_a                     : 246.550
p_a                     : 41.000

[Eficiências]
eta_inlet               : 0.850
eta_compressor          : 0.750
eta_camara              : 1.000
eta_turbina_compressor  : 0.900
eta_turbina_livre       : 0.900
eta_bocal_quente        : 0.980

[Gammas]
gamma_inlet             : 1.400
gamma_compressor        : 1.370
gamma_camara            : 1.350
gamma_turbina_compressor: 1.330
gamma_turbina_livre     : 1.330
gamma_bocal_quente      : 1.360

[Dados Operacionais]
prc                     : 15.770
pr_tl                   : 2.878
hydrogen_fraction       : 0.000
pressure_loss           : 0.000
kerosene_PCI            : 45000.000
hydrogen_PCI            : 120000.000
mean_R_air              : 288.300
Cp                      : 1.110
Cp_tl                   : 1.160
T04                     : 1600.000

[Dados da Gearbox e Hélice]
ge

In [5]:
turboprop.final_config

{'mach': 0.45,
 'altitude': 0.0,
 'eta_inlet': 0.85,
 'gamma_inlet': 1.4,
 'eta_compressor': 0.75,
 'gamma_compressor': 1.37,
 'eta_camara': 1,
 'gamma_camara': 1.35,
 'eta_turbina_compressor': 0.9,
 'gamma_turbina_compressor': 1.33,
 'eta_turbina_livre': 0.9,
 'gamma_turbina_livre': 1.33,
 'eta_bocal_quente': 0.98,
 'gamma_bocal_quente': 1.36,
 'prc': 15.77,
 'pr_tl': 2.8778,
 'hydrogen_fraction': 0.0,
 'pressure_loss': 0.0,
 'kerosene_PCI': 45000.0,
 'hydrogen_PCI': 120000.0,
 'mean_R_air': 288.3,
 'Cp': 1.11,
 'Cp_tl': 1.16,
 'T04': 1600,
 'gearbox_efficiency': 0.98,
 'propeller_efficiency': 0.85,
 'max_gearbox_power': 2050,
 'ref_pot_th': 2457,
 't_a': 246.55,
 'p_a': 41}

### Exemplo 7.3

In [6]:
turbofan = Turbofan(config_ex73)
turbofan.set_sea_level_air_flow(88.42)
turbofan.save_design_point()
turbofan.update_from_N2(0.85)
turbofan.print_outputs()

Salvando ponto de projeto...
--- Resultados da Simulação do Motor ---

[ Estações do Motor ]
Estação         | Temp. Total (K)      | Pressão Total (kPa) 
-------------------------------------------------------------
2 (Inlet)       | 256.535              | 46.920              
8 (Fan)         | 278.681              | 61.318              
3 (Compressor)  | 513.065              | 375.112             
4 (Câmara)      | 1269.711             | 375.112             
5 (Turbina Comp.) | 1035.326             | 145.702             
6 (Turbina Fan) | 901.123              | 76.415              

[ Velocidades de Saída ]
Velocidade de Voo (u_0)         : 141.955 m/s
Velocidade Bocal Quente (u_core): 540.634 m/s
Velocidade Bocal Frio (u_fan)   : 244.693 m/s

[ Performance Geral ]
Razão Combustível/Ar (f)        : 0.01932
Empuxo Específico               : 0.92898 kN.s/kg
Consumo Específico (TSFC)       : 0.02079 kg/(kN.s)
Empuxo Total                    : 4.473 kN
Consumo de Combustível          : 0